In [ ]:
# Pillow based operations
def resize_smallest_dim_with_ratio(width, height, target):
    """
        Resize the smallest dimension while keeping the aspect ratio.
    """
    if height == width:
        return target, target
    
    if height < width:
        rwidth = (width / height) * target
        return int(rwidth), target 
    
    if width < height:
        rheight = (height / width) * target
        return target, int(rheight)
    
    return width, height

def resize_single_smallest_with_ratio(image, target):    
    width, height = resize_smallest_dim_with_ratio(*image.size, target)
    image = image.resize((width, height), Image.BILINEAR)
    return image

In [ ]:
# Pillow based operations
def crop_largest_rectangle(width, height, verbose=False):
    if width == height:
        return 0, 0, width, height
    
    largest = width
    smallest = height
        
    if height > width:
        largest = height
        smallest = width
    
    to_crop = largest - smallest
    if verbose:
        print(to_crop)
        
    offset_largest_start = to_crop // 2
    offset_largest_end = largest - offset_largest_start
    if to_crop % 2 != 0:
        offset_largest_end = offset_largest_end - 1
    
    if width > height:
        return offset_largest_start, 0, offset_largest_end, height
    
    if height > width:
        return 0, offset_largest_start, width, offset_largest_end  
    
    return 0, 0, width, height    

def crop_single_largest(image, verbose=False):
    """
        Returns a copy of the image that has the largest dimension dropped
    """
    width, height = image.size
    
    image = image.copy()
    if verbose:
        print(image.size)   
        
    to_crop_box = crop_largest_rectangle(width, height)
    
    image = image.crop(box=to_crop_box)
    
    if verbose:
        print(image.size)
        
    return image

In [ ]:
# Pillow based operations
%matplotlib inline
from PIL import Image
from matplotlib import pyplot as plt
import numpy as np

def crop_central_single(image, offset, verbose=False):
    image = image.copy()
    # offset: total pixels to ignore in heigh and width
    offset_x = offset // 2
    offset_y = offset // 2
    if verbose:
        print(image.size)
        print(offset_x, offset_y)
    x, y = image.size # 256, 256: remove 16 on each edge
    
    crop_x = x - offset_x
    crop_y = y - offset_y
    if verbose:
        print(crop_x, crop_y)
    image = image.crop(box=(offset_x, offset_y, x - offset_x, y - offset_y))
    if verbose:
        print(image.size)
    return image

def resize_single(image, height, width, mode):    
    image = image.resize((width, height), Image.BILINEAR)
    return image

def read_single(image_path):
    image = Image.open(image_path)
    image = image.convert("L")  # greyscale
    return image

def read_single_rgb(image_path):
    image = Image.open(image_path)
    image = image.convert("RGB")
    return image

def save_single(image, path):
    image.save(path)
    
def show_single(image):
    image = np.asarray(image, dtype=np.int32)
    plt.axis("off")
    plt.imshow(image, cmap="gray")
    plt.show()

def show_single_rgb(image):
    image = np.asarray(image, dtype=np.uint8)
    plt.axis("off")
    plt.imshow(image)
    plt.show()
    
def show_single_row(images):
    # row-wise with dynamic number of columns
    max_cols = len(images)
    fig, axes = plt.subplots(nrows=1, ncols=max_cols, figsize=(20, 20))
    for idx, image in enumerate(images):
        image = np.asarray(image, dtype=np.int32)
        col = idx % max_cols
        axes[col].axis("off")
        axes[col].imshow(image, cmap="gray")
    plt.subplots_adjust(wspace=.05, hspace=.05)
    plt.show()

In [ ]:
# Sci-Kit images based operations
%matplotlib inline

from PIL import Image
from matplotlib import pyplot as plt

from skimage import exposure, img_as_ubyte
from skimage.transform import resize
from skimage import img_as_ubyte

import numpy as np

def crop_central_single(image, offset, verbose=False):
    # offset: total pixels to ignore in heigh and width
    offset_x = offset // 2
    offset_y = offset // 2
    if verbose:
        print(image.shape)
        print(offset_x, offset_y)
    y, x = image.shape # 256, 256: remove 16 on each edge
    
    crop_x = x - offset_x
    crop_y = y - offset_y
    if verbose:
        print(crop_x, crop_y)
    crop = image[offset_y:crop_y, offset_x:crop_x]
    if verbose:
        print(crop.shape)
    return crop

def crop_single(image, verbose=False):
    if verbose:
        print(image.shape)
    y, x = image.shape # 426, 640: remove 107 on width at each side   
    cropx = (x-y) // 2
    if verbose:
        print(cropx)
    return image[:,cropx:x-cropx]

def resize_single(image, height, width, mode):    
#    0: Nearest-neighbor
#    1: Bi-linear (default)
#    2: Bi-quadratic
#    3: Bi-cubic
#    4: Bi-quartic
#    5: Bi-quintic
    image = resize(image, (height, width), order=mode, preserve_range=True)
    image = np.floor(image)
    image = image.astype(np.int32)
    return image

def read_single(image_path):
    img = Image.open(image_path)
    img = img.convert("L")  # greyscale
    img_arr = np.asarray(img, dtype=np.int32)
    return img_arr

def save_single(image, path):
    im = Image.fromarray(image)
    im = im.convert("L")
    im.save(path)
    
def show_single(image):
    plt.axis("off")
    plt.imshow(image, cmap="gray")
    plt.show()
    
def show_single_row(images):
    # row-wise with dynamic number of columns
    max_cols = len(images)
    fig, axes = plt.subplots(nrows=1, ncols=max_cols, figsize=(20, 20))
    for idx, image in enumerate(images):
        col = idx % max_cols
        axes[col].axis("off")
        axes[col].imshow(image, cmap="gray")
    plt.subplots_adjust(wspace=.05, hspace=.05)
    plt.show()

In [ ]:
# Plot only images that are available on a grid (so no axis for non-image subplots)
import numpy as np
from matplotlib import pyplot as plt

def show_many(images, max_rows, max_cols, figsize=(20, 20)):
    fig = plt.figure(figsize=figsize, dpi=300)

    for idx, image in enumerate(images):
        image = np.squeeze(image)
        row = idx // max_cols
        col = idx % max_cols

        ax = plt.subplot2grid((max_rows, max_cols), (row, col))
        ax.axis("off")
        ax.imshow(image, cmap="gray", aspect="auto")
        plt.subplots_adjust(wspace=.05, hspace=.1)
        
    plt.show()

In [ ]:
# Plot only images that are available on a grid (so no axis for non-image subplots)
import numpy as np
from matplotlib import pyplot as plt

def show_many(images, max_rows, max_cols, figsize=(20, 20), titles = None, save=False, save_path=None):
    fig = plt.figure(figsize=figsize, dpi=300)

    for idx, image in enumerate(images):
        image = np.squeeze(image)
        row = idx // max_cols
        col = idx % max_cols

        ax = plt.subplot2grid((max_rows, max_cols), (row, col))
        if titles != None:
            ax.set_title(titles[idx])
        ax.axis("off")
        ax.imshow(image, cmap="gray", aspect="auto")
    if titles == None:
        plt.subplots_adjust(wspace=.05, hspace=.05)        
    else:
        plt.subplots_adjust(wspace=.05, hspace=.1)
        
    if save:
        plt.savefig(save_path)
        plt.close(fig)
    else:
        plt.show()